In [4]:
!pip install -e /home/sagar/modeling-info-spread

Obtaining file:///home/sagar/modeling-info-spread
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.2.8
    Uninstalling src-0.2.8:
      Successfully uninstalled src-0.2.8
  Running setup.py develop for src


In [5]:
import os
import matplotlib.pyplot as plt
from matplotlib import colormaps
from src.models.NSI import NSI
from src.models.channel import *
from tqdm.notebook import tqdm
import pickle
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from scipy import stats
from collections import defaultdict
import numpy as np

# Constants

In [6]:
N = 10000
T = 1000

beta = 0.01
k = 4

# Defining Channels and Models

# Calculating Numerical Results

# Running Simulations